In [46]:
import pandas as pd
import plotly.express as px
import re
import datetime

from Project.Database import Db

# Example data for the TPM

In [47]:
# pd.read_csv("/Users/christiannielsen/Documents/GitHub/Datavidenskab-P6/Project/TPM/Data/ISID.csv")

In [48]:
# pd.read_json(
#     path_or_buf="/Users/christiannielsen/Documents/GitHub/Datavidenskab-P6/Project/TPM/output/Experiment_minsup0.1_minconf_0.1/level1.json",
#     orient='name_node')

In [49]:
year1_minutes, meta = Db.load_data(hourly=False, meta=True, year=1)

In [50]:
appliance_condition = (lambda self: (
        (self["Parameter"] == "Status_OnOff") &
        (self["Subsystem"] == "Loads") &
        (~self["Unnamed: 0"].str.contains("Child")) &
        (~self["Unnamed: 0"].str.contains("Prnt")) &
        (~self["Unnamed: 0"].str.contains("StatusLatentload"))
))

appliances_statusOnOff = meta.loc[appliance_condition]["Unnamed: 0"].tolist()

In [51]:
#Replaces values to be binary 0 or 1.
for i in appliances_statusOnOff:
    year1_minutes.loc[year1_minutes[i] < 0, i] = 0
    year1_minutes.loc[year1_minutes[i] > 0, i] = 1

len(appliances_statusOnOff)

40

In [52]:
minute_appliances_status = year1_minutes[["Timestamp"] + appliances_statusOnOff].copy()

In [53]:
for col in meta["Measurement_Location"].unique():
    appliance_condition = (lambda self: (
            (self["Parameter"] == "Status_OnOff") &
            (self["Unnamed: 0"].str.contains("Light")) &
            (self["Measurement_Location"] == col)
    ))
    light_cols = meta.loc[appliance_condition, "Unnamed: 0"]
    if len(light_cols) > 1:
        minute_appliances_status[col + "_Lights"] = minute_appliances_status[light_cols].max(1)
        minute_appliances_status.drop(light_cols, axis= 1, inplace=True)
        # print(year1_minutes[col + "_Lights"])

In [ ]:
#Forces Timestamp to the type of datetime, to extract the hour of Timestamp.
minute_appliances_status["Timestamp"] = pd.to_datetime(minute_appliances_status["Timestamp"],
                                                       format="%Y-%m-%d %H:%M:%S%z", utc=True) - pd.to_timedelta(
    unit="h", arg=(
        minute_appliances_status["Timestamp"].str.split(pat="-", expand=True)[lambda self: self.columns[-1]].str[
            1]).astype(
        int))

#Extracts hour of Timestamp.
minute_appliances_status["DayOfYear"] = minute_appliances_status.Timestamp.dt.dayofyear

In [ ]:
minute_appliances_status['Timestamp'] = (pd.to_datetime(minute_appliances_status["Timestamp"],
                                                       format="%Y-%m-%d %H:%M:%S%z", utc=True).astype("int") / 1e9).astype("int")

In [ ]:
minute_appliances_status["DayOfYear"] = ((minute_appliances_status['Timestamp'] - minute_appliances_status['Timestamp'].min()) / (3600 * 24)).astype("int")

In [ ]:
minute_appliances_status

In [ ]:
appliance_job_list = []

for appliance in appliances_statusOnOff:
    name = " ".join(re.findall('[A-Z][^A-Z]*', appliance.split("Load_Status")[-1]))
    appliance_switch = minute_appliances_status[(minute_appliances_status[appliance] != minute_appliances_status[appliance].shift(1))][["Timestamp", "DayOfYear", appliance]][1:]

    df_indices = appliance_switch.index.tolist()
    for index, df_index in enumerate(df_indices):
        if appliance_switch.loc[df_index, appliance]:
            try:
                appliance_job_list.append({"start": appliance_switch.loc[df_index, "Timestamp"],
                                           "end": appliance_switch.loc[df_indices[index + 1], "Timestamp"],
                                           "appliance": name,
                                           "day": appliance_switch.loc[df_indices[index], "DayOfYear"]
                                           })
            except:
                continue

appliance_job_list

In [ ]:
df = pd.DataFrame(appliance_job_list).sort_values(by=['day', 'start']).reset_index(drop=True)
df

In [ ]:
df.to_csv(path_or_buf="C:\\Users\\Ronni\\Documents\\GitHub\\Datavidenskab-P6\\Project\\TPM\\TPM\\Data\\houseData.csv", header=False, index=False)

In [ ]:
level2=pd.read_json(path_or_buf="/Users/christiannielsen/Documents/GitHub/Datavidenskab-P6/Project/TPM/output/Experiment_minsup0.1_minconf_0.1/level2.json")

In [ ]:
level2.loc[level2["name_node"] == "Entry Hall Lights,Plug Load L R Blue Ray", "patterns"].values[0][0]["conf"]